In [371]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from math import exp

In [372]:
#подготовим наш датасет:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['class'] = iris.target
df = df[df['class'] != 0]
df['class'].replace([1, 2], [1, 0], inplace=True) 
result = pd.DataFrame(data=df['class'])
df.drop('class', axis=1, inplace=True)

In [373]:
# Реализуем функцию логистической регресии
def predict(row, coefficients):
    y = coefficients[0]
    for i in range(len(row)):
        y += coefficients[i + 1] * row[i]
    return 1.0 / (1.0 + exp(-y))

In [374]:
#модлеькой sklearn подберем лучшие коэффициенты:
clf = LogisticRegression().fit(df, result['class'])
clf.intercept_, clf.coef_

(array([14.43080433]),
 array([[ 0.39443136,  0.51327025, -2.93075043, -2.4170433 ]]))

In [375]:
# Подставим коэффициенты в нашу функцию лог. регрессии
result['predict'] = df.apply(predict, coefficients=[14.43,0.39,0.51,-2.93,-2.41], axis=1)

In [376]:
#Создадим функцию для проверки точности:
def calculate_accuracy(df):
    valuelist = df.values.tolist()
    correct_count = 0
    for element in valuelist:
        if element[1] > 0.5:
            predicted_value = 1
        else:
            predicted_value = 0
        if predicted_value == element[0]:
            correct_count+=1
    return correct_count / len(valuelist)

In [377]:
calculate_accuracy(result)

0.96

Как мы видим, с полученными коэффициентами наша регрессия работает с очень высокой точностью.

In [457]:
# # разработаем функцию градиентного спуска: 

# def coefficients_gd(train, l_rate, n_epoch):
#     #начнём с 0 точки:
#     coef = [0.0 for i in range(len(train[0]))]
#     for epoch in range(n_epoch):
#         sum_error = 0
#         for row in train:
#             y = predict(row[0:4], coef)
#             error = row[-1] - y
#             #Ошибку посчитаем методом MSE             
#             sum_error += error**2
#             coef[0] = coef[0] + l_rate * error * y * (1.0 - y)
#             for i in range(len(row)-1):
#                 coef[i + 1] = coef[i + 1] + l_rate * error * y * (1.0 - y) * row[i]
#     return coef

In [476]:
# разработаем функцию градиентного спуска: 

def coefficients_gd(train, l_rate, n_epoch):
    #начнём с 0 точки:
    coef = [0.0 for i in range(len(train[0]))]
    for epoch in range(n_epoch):
        sum_error = 0
        for row in train:
            y = predict(row[0:4], coef)
            error = row[-1] - y
            #Ошибку посчитаем методом MSE             
            sum_error += error**2
            coef[0] = coef[0] + l_rate * y * (1.0 - y) * error
            for i in range(len(row)-1):
                coef[i + 1] = coef[i + 1] + l_rate * y * (1.0 - y) * row[i] * error
    return coef

In [478]:
dft = df.copy()
dft['class'] = result['class']
data=dft.values.tolist()

In [479]:
l_rate = 0.2
n_epoch = 1000
coef = coefficients_gd(data, l_rate, n_epoch)
print(coef)

[8.634036388917055, 5.445003851939993, 6.566652876848133, -8.641267875165145, -12.723017457111949]


In [480]:
result['predict'] = df.apply(predict, coefficients=coef, axis=1)

In [481]:
calculate_accuracy(result)

0.94

In [ ]:
C помощью градиентного спуска с низким LR и большим количеством итераций удалось приблизться к такому же качеству модели, которое выдает sklearn